In [ ]:
from typing import Callable

# Functions 

In [4]:
def fib(n: int):    # write Fibonacci series less than n
    """Print a Fibonacci series less than n."""
    a, b = 0, 1
    while a < n:
        print(a, end=' ')
        a, b = b, a+b
    print()

fib(2000)
print(fib.__doc__)
print(fib.__annotations__)

0 1 1 2 3 5 8 13 21 34 55 89 144 233 377 610 987 1597 
Print a Fibonacci series less than n.
{'n': <class 'int'>}


## Scope & Namespaces

### global & nonlocal keywords

In [ ]:
x = "global value"

def use_global():
    global x
    x = "modified globally"

use_global()
print(x)  # modified globally

def outer():
    msg = "enclosing value"
    def inner():
        nonlocal msg
        msg = "modified enclosing"
    inner()
    return msg

print(outer())  # modified enclosing

## Arguments

In [ ]:
def parrot(voltage, state='a stiff', action='voom', type='Norwegian Blue'):
    print("-- This parrot wouldn't", action, end=' ')
    print("if you put", voltage, "volts through it.")
    print("-- Lovely plumage, the", type)
    print("-- It's", state, "!")
    
parrot(1000)                                          # 1 positional argument
parrot(voltage=1000)                                  # 1 keyword argument
parrot(voltage=1000000, action='VOOOOOM')             # 2 keyword arguments
parrot(action='VOOOOOM', voltage=1000000)             # 2 keyword arguments
parrot('a million', 'bereft of life', 'jump')         # 3 positional arguments
parrot('a thousand', state='pushing up the daisies')  # 1 positional, 1 keyword

# parrot()                     # required argument missing
# parrot(voltage=5.0, 'dead')  # non-keyword argument after a keyword argument
# parrot(110, voltage=220)     # duplicate value for the same argument
# parrot(actor='John Cleese')  # unknown keyword argument

### Parameter Types

-  Syntax: [pos-only params] / , [pos-or-kw params] , * , [kw-only params]

In [ ]:
def f_pos_only(a, b, /):
    return a + b

def f_kw_only(*, sep=" "):
    return f"spam{sep}eggs"

def f_mixed(a, /, b, *, c=0):
    return a + b + c

print(f_pos_only(1, 2))          # 3
print(f_kw_only(sep="-"))        # spam-eggs
print(f_mixed(1, 2, c=3))        # 6

### Arbitrary arguments

In [7]:
# Note: #arguments recieves a tuple containing the positional arguments and no keyword arguments
# Note: **keywords recieves a dictionary containing all keyword arguments and no positional arguments
def cheeseshop(kind, *arguments, **keywords):
    print("-- Do you have any", kind, "?")
    print("-- I'm sorry, we're all out of", kind)
    for arg in arguments:
        print(arg)
    print("-" * 40)
    for kw in keywords:
        print(kw, ":", keywords[kw])
        
cheeseshop("Limburger", "It's very runny, sir.",
           "It's really very, VERY runny, sir.",
           shopkeeper="Michael Palin",
           client="John Cleese",
           sketch="Cheese Shop Sketch")

-- Do you have any Limburger ?
-- I'm sorry, we're all out of Limburger
It's very runny, sir.
It's really very, VERY runny, sir.
----------------------------------------
shopkeeper : Michael Palin
client : John Cleese
sketch : Cheese Shop Sketch


## Returns


In [ ]:
def divmod_pair(a: int, b: int):
    """Return quotient and remainder as a tuple."""
    return a // b, a % b

q, r = divmod_pair(17, 5)
print(q, r)  # 3 2

### Multiple Return Values (tuple packing/unpacking)

In [ ]:

def min_max_sum(nums):
    mn = min(nums) if nums else None
    mx = max(nums) if nums else None
    s = sum(nums)
    return mn, mx, s

mn, mx, s = min_max_sum([3, 1, 4])
print(mn, mx, s)  # 1 4 8

In [ ]:
# Functions without explicit return -> return None
def just_print(x):
    print(">>", x)

print(just_print("hello"))  # None

## Closures

In [4]:
def make_multiplier(factor: int) -> Callable[[int], int]:
    def mul(x: int) -> int:
        return x * factor
    return mul

times3 = make_multiplier(3)
print(times3(10))  # 30

30


In [ ]:
# Capturing state with a factory
def make_counter(start: int = 0) -> Callable[[], int]:
    count = start
    def inc():
        nonlocal count
        count += 1
        return count
    return inc

c1 = make_counter()
print(c1(), c1(), c1())  # 1 2 3

## Lambda Functions

### lambda

In [9]:
def make_incrementor(n):
    return lambda x: x + n

f = make_incrementor(42)
f(0)

42

### map()

In [ ]:
double = lambda x: x * 2
print(double(21))

print(list(map(lambda x: x*x, [1,2,3,4])))  # [1, 4, 9, 16]

### filter()

In [ ]:
print(list(filter(lambda x: x%2==0, range(10))))  # [0, 2, 4, 6, 8]

### sorted()

In [ ]:
words = ["pear","fig","apple","banana"]
print(sorted(words, key=lambda s: (len(s), s)))  # by length then lexicographic

## Decorators

### Simple validation decorator

In [ ]:
from functools import wraps

def nonempty_args(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        if not args and not kwargs:
            raise ValueError("At least one argument is required")
        return fn(*args, **kwargs)
    return wrapper

@nonempty_args
def join_with_dash(*parts):
    return "-".join(map(str, parts))

print(join_with_dash("a","b","c"))  # a-b-c

In [ ]:
import time
from functools import wraps

def timed(fn):
    @wraps(fn)
    def wrapper(*args, **kwargs):
        t0 = time.time()
        try:
            return fn(*args, **kwargs)
        finally:
            dt = (time.time() - t0) * 1000
            print(f"{fn.__name__} took {dt:.2f} ms")
    return wrapper

@timed
def slow_add(a, b):
    time.sleep(0.01)
    return a + b

print(slow_add(1, 2))  # prints timing, then 3

In [ ]:
from functools import wraps
# parametrized
def retries(n: int):
    def deco(fn):
        @wraps(fn)
        def wrapper(*args, **kwargs):
            last = None
            for _ in range(n):
                try:
                    return fn(*args, **kwargs)
                except Exception as e:
                    last = e
            # only raise if always failed
            raise last
        return wrapper
    return deco

@retries(3)
def maybe_fail(x: int):
    if x < 0:
        raise ValueError("bad")
    return x

print(maybe_fail(5))   # 5
# print(maybe_fail(-1))  # would raise after 3 tries

## Recursion

In [10]:
def factorial(n):
    """Return the factorial of n."""
    if n == 0:
        return 1
    return n * factorial(n - 1)

print(factorial(5))  # Output: 120

120
